In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

# Load dataset
df = pd.read_csv("/Users/ameen/Documents/M.tech/CBDP/ML/SAMPLE/pg_admissions_dataset.csv")

# Select key features
features = ['UG_percentage', 'GATE_score', 'Work_exp_years']
target = 'program_id'

df = df.dropna(subset=features + [target])

# Encode categorical columns
enc = LabelEncoder()
df[target] = enc.fit_transform(df[target])
num_classes = len(enc.classes_)

X = df[features].values
y = df[target].values

# Scale numerical data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to torch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = x.unsqueeze(1)
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])
        return out

model = LSTMModel(input_dim=3, hidden_dim=32, output_dim=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 500
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        _, predicted = torch.max(outputs, 1)
        acc = (predicted == y_train).float().mean()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f},Train Acc: {acc.item()*100:.2f}")

# Save model, encoder, and scaler
torch.save({
    'model_state_dict': model.state_dict(),
    'encoder_classes': enc.classes_,
    'scaler_mean': scaler.mean_,
    'scaler_scale': scaler.scale_
}, '/Users/ameen/Documents/M.tech/CBDP/ML/SAMPLE/lstm_model3.pth')

print("✅ Model trained and saved successfully.")


Epoch [10/500], Loss: 2.9928,Train Acc: 6.54
Epoch [20/500], Loss: 2.9890,Train Acc: 7.36
Epoch [30/500], Loss: 2.9856,Train Acc: 7.08
Epoch [40/500], Loss: 2.9822,Train Acc: 7.63
Epoch [50/500], Loss: 2.9788,Train Acc: 7.36
Epoch [60/500], Loss: 2.9755,Train Acc: 7.22
Epoch [70/500], Loss: 2.9721,Train Acc: 6.54
Epoch [80/500], Loss: 2.9687,Train Acc: 7.22
Epoch [90/500], Loss: 2.9652,Train Acc: 7.49
Epoch [100/500], Loss: 2.9618,Train Acc: 8.04
Epoch [110/500], Loss: 2.9583,Train Acc: 7.90
Epoch [120/500], Loss: 2.9549,Train Acc: 7.63
Epoch [130/500], Loss: 2.9516,Train Acc: 7.90
Epoch [140/500], Loss: 2.9483,Train Acc: 8.45
Epoch [150/500], Loss: 2.9452,Train Acc: 8.99
Epoch [160/500], Loss: 2.9422,Train Acc: 9.13
Epoch [170/500], Loss: 2.9393,Train Acc: 8.86
Epoch [180/500], Loss: 2.9365,Train Acc: 9.40
Epoch [190/500], Loss: 2.9336,Train Acc: 9.81
Epoch [200/500], Loss: 2.9308,Train Acc: 9.81
Epoch [210/500], Loss: 2.9278,Train Acc: 9.81
Epoch [220/500], Loss: 2.9248,Train Acc: 9.